### Introduction

In [15]:
import pandas as pd

### Regression

#### Model

#### Analysis

### Neural Network 

The linear regression yielded poor results, so we thought a more complex regression might lend towards a better analysis. For this we decided to build a neural network using Tensorflow. We chose a feedforward neural network due to its ability to model non linear functions.

We begin by importing the Tensorflow module, which provides a framework for, what seems like, every type of modeling technique. Take note that 'tf' is the industry standard alias for tensorflow.

In [16]:
import tensorflow as tf

Next, we read in the balanced song features dataset, which includes data on every song that has appeared on the Billboard Hot 100 since 2010. Intially the dataset was split such that 70% of the music was unpopular and 30% popular. After completing the training the network had an outstanding 70% accuracy! Under further investigation, we realized the networks was simply assigning everything an unpopular label. Do remedy this we modified our dataset such that there was a 50-50 split between popular and unpopular songs. 

In [17]:
#Read in File
data = pd.read_csv("balanced_pop_unpop_features.csv")

#Remove track name column
del data["Unnamed: 0"]

#Quick look at the data
data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,label
0,0.21400,0.666,178242,0.677,0.000000,2,0.0979,-5.743,0.0326,100.014,4,0.178,1
1,0.01340,0.807,183750,0.916,0.000012,0,0.0787,-3.282,0.2260,127.973,4,0.651,1
2,0.00162,0.791,279507,0.615,0.000065,6,0.0812,-6.149,0.0667,128.017,4,0.393,1
3,0.76300,0.707,275227,0.709,0.000000,11,0.2740,-3.979,0.3400,89.094,4,0.501,1
4,0.57000,0.629,250173,0.572,0.000000,5,0.1920,-7.733,0.0387,100.015,4,0.386,1


In [28]:
print(len(data.columns))

13


The Tensorflow software below automatically shuffles the data, but just to be sure the following line of code mixes up the rows of the dataframe.


In [18]:
#Shuffle dataset
for i in range(10): data = data.sample(frac=1)
    
#Quick look at the shuffled data
data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,label
1032,0.3310,0.525,193893,0.477,0.000000,10,0.1110,-5.453,0.0270,127.106,4,0.339,0
633,0.0145,0.716,215253,0.972,0.000032,7,0.3170,-2.302,0.1960,110.026,4,0.576,0
542,0.0113,0.834,206813,0.758,0.000000,11,0.0238,-7.135,0.1060,98.021,4,0.759,0
5,0.2260,0.718,222200,0.801,0.000000,0,0.3900,-2.581,0.0386,127.016,4,0.538,1
457,0.3570,0.858,199440,0.799,0.000000,2,0.0910,-4.034,0.0589,121.991,4,0.966,1


#### Model 

To model the data, the data is split 70-30, where 70% of the data is used to train the model and 30% of the data is used to test the model. 

In [19]:
#About a 70-30 split between training and testing 
train = data.iloc[:750]
test = data.iloc[750:]

#Quick look at shape of the two sets
print(train.shape,test.shape)

(750, 13) (322, 13)


Next, the training and test sets are split into sets of features and labels. Immedietly following, the sets are normalized using a Z-score.

In [20]:
#Split entries in train_x: features and train_y: labels
train_x = train.iloc[:,:12]
train_y = train.iloc[:,12:].astype(int)

test_x = test.iloc[:,:12]
test_y = test.iloc[:,12:].astype(int)

#Take Z-score
train_x = (train_x-train_x.mean())/(train_x.std())
test_x = (test_x-test_x.mean())/(test_x.std())

#Quick look at training set
train_x.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
1032,0.859973,-0.874249,-0.686885,-1.285935,-0.115799,1.231625,-0.489342,0.179266,-0.719090,0.211510,0.130038,-0.776372
633,-0.745659,0.510739,-0.160687,1.748438,-0.115302,0.420397,0.920337,1.542387,1.002004,-0.406188,0.130038,0.273416
542,-0.761893,1.366386,-0.368604,0.436608,-0.115799,1.502034,-1.086061,-0.548366,0.085445,-0.840349,0.130038,1.084011
5,0.327299,0.525241,0.010451,0.700200,-0.115799,-1.472470,1.419884,1.421692,-0.600956,0.208255,0.130038,0.105095
457,0.991873,1.540416,-0.550236,0.687940,-0.115799,-0.931651,-0.626204,0.793125,-0.394221,0.026526,0.130038,2.000914


Tensorflow requires that a list of column features be provided in order to specify the type of input. The neural network is capable of using other feature types other than numeric.

In [30]:
#Create feature column for net 
feature_columns = []
for i in train_x.columns:
    feature_columns.append(tf.feature_column.numeric_column(key=i))

#Little preview of the numeric_column object
for i in feature_columns[:2]:
    print(i,"\n")

_NumericColumn(key='acousticness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) 

_NumericColumn(key='danceability', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) 



The following code is used to create the neural network. We decided to use a single hidden layer with 5 nodes.

In [22]:
#Make estimator/ Model

my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 4,  # Save checkpoints every 20 minutes.
    keep_checkpoint_max = 10,       # Retain the 10 most recent checkpoints.
)

classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns
                                        , hidden_units=[5],
                                        n_classes=2,model_dir="./modeldata",
                                       config=my_checkpointing_config)

INFO:tensorflow:Using config: {'_model_dir': './modeldata', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 4, '_session_config': None, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1821846048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


The following function is used to prepare the data for input to the neural network. It is covered in depth in the Tensorflow documentation.

In [23]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

Next, the network is training using the entirety of the training set. The Batch size is 500 and the number of steps or epochs is 1500.

In [27]:
# Train the Model.
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y,300),
    steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./modeldata/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into ./modeldata/model.ckpt.
INFO:tensorflow:loss = 188.96317, step = 1001
INFO:tensorflow:global_step/sec: 214.972
INFO:tensorflow:loss = 194.00017, step = 1101 (0.466 sec)
INFO:tensorflow:global_step/sec: 271.233
INFO:tensorflow:loss = 194.0174, step = 1201 (0.369 sec)
INFO:tensorflow:global_step/sec: 285.102
INFO:tensorflow:loss = 189.95126, step = 1301 (0.351 sec)
INFO:tensorflow:global_step/sec: 285.693
INFO:tensorflow:loss = 193.3042, step = 1401 (0.350 sec)
INFO:tensorflow:global_step/sec: 282.97
INFO:tensorflow:loss = 186.79214, step = 1501 (0.353 sec)
INFO:tensorflow:global_step/sec: 291.454
INFO:tensorflow:loss = 196.06117, step = 

#### Analysis

Now that the network is trained it is time to test it with the testing set. The following function prepares the testing set for testing. 

In [25]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

The following code runs the testing evaluate function on the network.

In [26]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, 3000))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-14-21:45:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./modeldata/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-14-21:45:52
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.48447204, accuracy_baseline = 0.5031056, auc = 0.48572528, auc_precision_recall = 0.5003184, average_loss = 0.7447384, global_step = 1000, label/mean = 0.5031056, loss = 239.80576, precision = 0.48591548, prediction/mean = 0.49046984, recall = 0.42592594


Here are the results of the evaluation more clearly.

In [13]:
eval_result

{'accuracy': 0.57763976,
 'accuracy_baseline': 0.5031056,
 'auc': 0.59544754,
 'auc_precision_recall': 0.60107565,
 'average_loss': 0.68200713,
 'global_step': 3000,
 'label/mean': 0.5031056,
 'loss': 219.6063,
 'precision': 0.5802469,
 'prediction/mean': 0.496302,
 'recall': 0.5802469}

A print out of the prediction and exprected results can be found in the ANN Tests document located in the Model folder as well as a series of test done with other features and varying network shapes. 

From the evaluator it's clear that the neural network does a pretty bad job of modeling the data. We believe that to predict whether a song will be popular or not requires atleast some data on the cultural climate at the time. After reading a series of documents written by people conducting the same test, they also agree that analyzing a song's features is insufficient for producing meaningfull predictions. 

### Conclusion